In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import itertools
import matplotlib
from time import time
# from pandas.tools.plotting import table
import time
import os

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC

from sklearn.metrics import f1_score, adjusted_rand_score, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from mpl_toolkits.mplot3d import Axes3D
from pprint import pprint
from tqdm import tqdm
from copy import copy
from sklearn.decomposition import PCA, TruncatedSVD, SparsePCA
from sklearn.preprocessing import normalize

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KernelDensity
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix

from sklearn.decomposition import PCA, NMF, IncrementalPCA, FastICA, LatentDirichletAllocation, TruncatedSVD
from sklearn.manifold import Isomap, MDS, LocallyLinearEmbedding, TSNE

matplotlib.rcParams.update({'font.size': 16})

# plt.style.use("ggplot")
plt.style.use('default')


In [2]:
category = {}
category_sub = {}
description = {}

for nm in ["alphabay", "poseidon", "silkroad"]:
    description[nm], category[nm], category_sub[nm] = pickle.load(open("data/meta/{}_dataset.p".format(nm), "rb"))    
    print(nm, ":", len(description[nm]), "samples, \n", len(set(category[nm])),"mains, ", len(set(category_sub[nm])), "subs")

alphabay : 15222 samples, 
 12 mains,  48 subs
poseidon : 3688 samples, 
 10 mains,  36 subs
silkroad : 2137 samples, 
 1 mains,  22 subs


In [3]:
#
# reduce dimension
# Input: input, output_dimension 
# Out:   X, reduction_object
#


def red_dim(inp, dim_out, domain_bypass=False, norm=False):
    master_ = 3000
    for name in os.listdir("data/meta/dim_reduce"):
        
        if domain_bypass:
            dom = domain_bypass
        else:
            dom = domain
                    
        if "svd_{}_".format(dom) in name:            
            prev_vectorizer, svd = pickle.load(open("data/meta/dim_reduce/{}".format(name), "rb"))    

            if str(vectorizer.get_params) == str(prev_vectorizer):
                print("Pre-Trained SVD found: {}".format(name))
                pca = svd
                X = pca.transform(inp)
                X = X[:,range(dim_out)]
                if norm:
                    X = normalize(X, axis=0)
                return X, pca

    pca = TruncatedSVD(n_components=master_, random_state=1)
    pca.fit(inp)
    
    fn_ = "data/meta/dim_reduce/svd_{}_{}_{}.bow".format(domain, inp.shape[1], int(time.time()))
    
    pickle.dump((vectorizer.get_params, pca), open(fn_,"wb"))
    print("Saving SVD: {}".format(fn_))
    
    X = pca.transform(inp)
    X = X[:,range(dim_out)]
    if norm:
        X = normalize(X, axis=0)
    return X, pca

In [4]:
# print(input_data.shape)
# # input_, pca = red_dim(input_data, 10 , domain_bypass='poseidon')
# input_, pca = lda(input_data, 3) #, domain_bypass='poseidon')
# print(input_.shape)
# print(input_[0])

In [5]:
#
# reduce dimension
# Input: input, output_dimension 
# Out:   X, reduction_object
#

def lda(inp, out_dim=False, mid_dim=3000, domain_bypass=False, norm=True):
    if domain_bypass:
        dom = domain_bypass
    else:
        dom = domain
                
    mid_,_ = red_dim(inp, mid_dim, domain_bypass=dom)

    # CHECK FOR PRE-TRAINED LDAs
    for name in os.listdir("data/meta/dim_reduce"):
        if "lda_{}_{}_".format(dom, mid_dim) in name:
            prev_vectorizer, lda = pickle.load(open("data/meta/dim_reduce/{}".format(name), "rb"))    
            if str(vectorizer.get_params) == str(prev_vectorizer):
                print("Pre-Trained LDA found: {}".format(name))
                pca = lda
                X = pca.transform(mid_)
                if out_dim:
                    X = X[:,range(out_dim)]
                if norm:
                    X = normalize(X, axis=0)
                return X, lda

    # TRAIN NEW LDA    
    pca = LinearDiscriminantAnalysis()

    X = pca.fit_transform(mid_, problem_numeric[:inp.shape[1]])
    od = X.shape[1]

    fn_ = "data/meta/dim_reduce/lda_{}_{}_{}.bow".format(domain,
                                                            mid_dim,
                                                            int(time.time()))
    pickle.dump((vectorizer.get_params, pca), open(fn_,"wb"))
    print("Saving LDA: {}".format(fn_))
    if out_dim:
        X = X[:,range(out_dim)]
    if norm:
        X = normalize(X, axis=0)
    return X, pca

In [6]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

# vect = CountVectorizer(tokenizer=LemmaTokenizer())  

In [21]:
#
# TFIDF w/o using countvectorizer (Equivalent to CountVectorizer + TfidfTransformer)
#
def tfidf(inp):
    global vectorizer
    
    vectorizer = HashingVectorizer(n_features=150000)
#     vectorizer = HashingVectorizer(n_features=250000, non_negative=True)
#     inp = vectorizer.fit_transform(inp)
#     vectorizer = TfidfTransformer(sublinear_tf=False, use_idf=True)
    
#                                   token_pattern=u'\\w[A-Za-z]{3,10}\\b')
    
#     vectorizer = CountVectorizer(
#                                 min_df=2, 
#                                 max_df = 0.5,
#                                 token_pattern=u'\\w[A-Za-z]{3,10}\\b',
#                                 ngram_range=(1,2)
#                                 )
    
#     vectorizer = TfidfVectorizer(use_idf=True,
#                                 stop_words='english',
#                                 min_df = 3,
#                                 max_df= 0.5,
# #                                     vocabulary=voce,
#                                 sublinear_tf=True)
    
    ans = vectorizer.fit_transform(inp)
    return ans

# input_data = tfidf(description[domain])

In [8]:
#
# clf = list of estimators
#

def create_clf(inp_, domain_bypass=False, pre_trained=True):

    # construct the estimators
    clf = VotingClassifier([
                            ('Support Vector Classifier (RBF)',SVC()),
                            ('Naive Bayes Bernoulli',sklearn.naive_bayes.BernoulliNB()),
                            ('Ridge Classifier',sklearn.linear_model.RidgeClassifier()),
                            ('K-Nearest Neighbors',sklearn.neighbors.KNeighborsClassifier()),
                            ('Random Forest Classifier', RandomForestClassifier()),
        
                            ('Support Vector Classifier (Linear)', SVC(C=0.01, 
                                                             decision_function_shape='ovo', 
                                                             gamma=1, 
                                                             kernel='linear')),
        
                            ('Stochastic Gradient Descent', SGDClassifier(alpha=0.001,
                                                                        epsilon=0.01,
                                                                        l1_ratio=0.1,
                                                                        loss='hinge',
                                                                        penalty='l1')),
        
                            ('Multi Layer Perceptron (SGD)', MLPClassifier(alpha=0.0001,
                                                      solver='sgd')),
                            
                            ('Multi Layer Perceptron (Adam)', MLPClassifier(alpha=0.0001,
                                                      solver='adam')),
        
                            ('Multi Layer Perceptron (Adam)',  MLPClassifier(solver='adam',
                                                                            hidden_layer_sizes=(200,100)
                                                                            ))
                           ])

    #
    # Check if already trained
    #
    
    if domain_bypass:
        dom = domain_bypass
    else:
        dom = domain
    
    if pre_trained:
        for name in os.listdir("data/meta/clf"):
            if "{}_{}".format(dom, inp_.shape[1]) in name:
                prev_vector, tmp_clf = pickle.load(open("data/meta/clf/{}".format(name), "rb"))
                if str(vectorizer.get_params) == str(prev_vector):
                    if str(tmp_clf.get_params) == str(clf.get_params):
                        print("Pre-Trained CLF found:" + name)
                        return tmp_clf
    
    
#     if pre_trained:
#         for name in os.listdir("data/meta/clfs"):
#             if str(inp_.shape[1]) in name:
#                 dom, data_shape, prev_vector, tmp_clf = pickle.load(open("data/meta/clfs/{}".format(name), "rb"))

#                 if domain == dom or domain_bypass == dom:
#                     print(":::{} & {}".format(data_shape, inp_.shape))
#                     if str(data_shape[1]) == str(inp_.shape[1]):
#                         if str(vectorizer.get_params) == str(prev_vector):
#                             if str(tmp_clf.get_params) == str(clf.get_params):
#                                 print("Pre-Trained CLF found:" + name)
#                                 return tmp_clf
    

    #
    # Train new - if none exist
    #
    X_train, X_test, y_train, y_test = train_test_split(inp_, problem_numeric, test_size=0.25, random_state=1)
    clf.fit(X_train, y_train)

    pickle.dump((vectorizer.get_params, clf), open("data/meta/clf/{}_{}_{}.clf".format(domain, 
                                                                                        inp_.shape[1],
                                                                                        int(time.time())), 
                                                   "wb"))
    
    print("Saving CLF as {}_{}_{}.clf".format(domain, inp_.shape[1], int(time.time())))

    return(clf)

#
# ---
#

# input_ = input_data
# clf = create_clf(input_)
# clfD = (clf, input_, "none", "{} - Default".format(input_.shape[1]))

# clfs = [clfD]
# #clfs = []
# for x in tqdm([1000, 300, 5, 3]):
#     input_, pca = red_dim(input_data, x)
#     clf = create_clf(input_)
#     clfs.append((clf, input_, pca, "{}dim".format(x)))

In [9]:
#
# Plot result of multiple classifiers (clfs)
#


def plot_clfs(clf_list, best_line=False):    
    %matplotlib inline
    best_score = 0
    
    fig = plt.figure(figsize=(25,7))
    matplotlib.rcParams.update({'font.size': 27})

    for ind, clf_tup in tqdm(enumerate(clf_list)):
        clf_, input_, pca_, names = clf_tup
        _, X_test, _, y_test = train_test_split(input_, problem_numeric, test_size=0.25, random_state=1)
        
        clf_count = np.arange(len(clf_.estimators))
        clf_scores = [est.score(X_test, y_test) for est in clf_.estimators_]
        
        clf_names = [n[0] for n in clf_.estimators]
        plt.bar(clf_count+((.8/len(clf_list)))*ind-.35, clf_scores, width=.8/(len(clf_list)), label=names)

        plt.xticks(clf_count, clf_names)
    
        if max(clf_scores)>best_score:
            best_score = max(clf_scores)
        
    if best_line:
        plt.axhline(y=best_score)
    
    fig.autofmt_xdate()

#     plt.legend(ncol=3)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title("{}\n{}%".format(domain, round(best_score*100, 1)))
    plt.ylim([0,1])
    plt.xlabel("Estimator")
    plt.ylabel("Accuracy")
    fig.show()
    plt.show()

# plot_clfs(clfs)

In [10]:
#
# F1 score measure
#

def eval_clf(clf_inp):    
    %matplotlib inline
    matplotlib.rcParams.update({'font.size': 16})

    classifier, input_, pca_, names = clf_inp

    X_train, X_test, y_train, y_test = train_test_split(input_, problem_numeric, test_size=0.25, random_state=1)

    fig = plt.figure(figsize=(12,8))
    clf_count = len(classifier.estimators)

    for i, clfs_ in tqdm(enumerate(classifier.estimators_)):
        y_pred = clfs_.predict(X_test)

        scores = []
#         scores.append(clfs_.score(X_test, y_test))
        for avg in ['weighted', 'micro', 'macro']:
            scores.append(f1_score(y_test, y_pred, labels=list(set(problem_numeric)), average=avg))

        plt.bar(np.arange(3)+(.1*i)-0.25, scores, width=.1, label=classifier.estimators[i][0])
        plt.title(names)

    plt.xticks(range(3), ['F1 Weighted', 'F1 Micro', 'F1 Macro'])
#     plt.legend(loc='upper left',ncol=2)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    fig.autofmt_xdate()
    plt.ylim([0,1])
    plt.xlabel("Evaluation method")
    plt.ylabel("Accuracy")
    fig.show()
    plt.show()
    
    
# for clf_ in clfs:
#     eval_clf(clf_)

In [11]:
#
# Plot confussion matrix
#


def plot_confusion_matrix(clf, normalize=False, estimator=None):    
    %matplotlib inline
    matplotlib.rcParams.update({'font.size': 16})

    classes = sorted(list(set(problem)))
    classifier, input_, pca_, names = clf
    if estimator:
        names = classifier.estimators[estimator][0] + " : " + names
        classifier = classifier.estimators_[estimator]
        
    X_train, X_test, y_train, y_test = train_test_split(input_, problem_numeric, test_size=0.25, random_state=1)
    
    y_pred = classifier.predict(X_test)
    print("Accuracy:",classifier.score(X_test, y_test))
 
    cm = confusion_matrix(y_test, y_pred)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title = 'Normalized confusion matrix \n {}'.format(names)
    else:
        title = 'Confusion matrix \n {}'.format(names)
        
    fig = plt.figure()
    ax = fig.add_subplot(111)
    fig.set_size_inches(10, 10) #(22, 14)

    cmap = plt.cm.Blues
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0., vmax=cm.max())
    plt.title(title)
#     plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    thresh = cm.max() * 0.85
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if cm[i, j] < 0.01:
            continue
        if normalize:
            thresh = .64
            val = str(round(cm[i, j]*100,2)).split(".")[0]+'%'
        else:
            val = cm[i, j]
            
        if cm[i, j] > thresh:
            cl="white"
        else:
            cl="black"
            
        plt.text(j, i, val, horizontalalignment="center", color=cl) #, alpha=cm[i,j]

#     plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
# plot_confusion_matrix(clfs[0])
# plot_confusion_matrix(clfs[0], normalize=True)

In [12]:
#
# Plot X (3D)  
# Input: X (global)
#

def plot_data_3d(text=False, specific_cat=False):
    %matplotlib inline
    
    matplotlib.rcParams.update({'font.size': 16})

#     X3d,_ = red_dim(input_data, 3)
    X3d,_ = lda(input_data)
    
    fig = plt.figure(1, figsize=(13, 10))
    ax = Axes3D(fig)
    y = problem
    for name, label in zip(list(set(problem)), list(set(y))):  
        if specific_cat!=False:
            if not name in specific_cat:
                continue
                
        ax.scatter(X3d[[label==t for t in y], 0], 
                  X3d[[label==t for t in y], 1],
                  X3d[[label==t for t in y], 2], 
                   label=name, 
                   s=25,
                   marker = 'o',
#                    c='B',
#                    marker = '+',
                   alpha=1)
   
        if text:
            ax.text3D(X3d[[label==t for t in y], 0].mean(),
                      X3d[[label==t for t in y], 1].mean(),
                      X3d[[label==t for t in y], 2].mean(), name)          

    plt.title(domain.capitalize()+"\n")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#     plt.legend(loc='upper left', ncol=2)
    ax.set_xlabel('\nPCA 0')
    ax.set_ylabel('\nPCA 1')
    ax.set_zlabel('\nPCA 2')
    
## Poseidon
#     plt.xlim([10, 20])
#     plt.ylim([-5, 5])
## Alphabay  
#     plt.xlim([-8, 4]) #9
#     plt.ylim([-7, 20])
#     ax.set_zlim([-21, 5])
    
    plt.show()
    return(X3d)

In [13]:
# plot_data_3d(text=False)

In [14]:
# plot_data_3d(x33, text=False, specific_cat='Weapons')

In [15]:
#
# Plot X (2D)  
# Input: X2d (global)
#


def plot_data_2d(text=False, specific_cat=False):
    %matplotlib inline
    
    matplotlib.rcParams.update({'font.size': 16})

#     X2d,_ = red_dim(input_data, 2)
    X2d,_ = lda(input_data)

    fig = plt.figure(figsize=(10,8))
    y = problem
    
    for name in list(set(y)):
        if specific_cat!=False:
            if not name in specific_cat:
                continue

        plt.scatter(X2d[[name==t for t in y], 0],
                    X2d[[name==t for t in y], 1],
                    marker = '+',
                    label=name, 
#                     c='r',
                    s=25,
                    alpha=1,
                   )
        if text:
            plt.text(X2d[[name==t for t in y], 0].mean(),
                     X2d[[name==t for t in y], 1].mean(),
                     name)
    plt.title(domain.capitalize())
    plt.xlabel("PCA 0")
    plt.ylabel("PCA 1")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#     plt.xlim([0, 125])
#     plt.ylim([-20, 80])
#     plt.xlim([-.025, .025])
#     plt.ylim([-.025, .1])
    
    plt.show()
    return X2d

# plot_data_2d()

# for x in set(category["alphabay"]):
#     print(x)
#     plot_data_2d(text=False, specific_cat=x)

In [16]:
# data2 = plot_data_2d(text=False)
# data2 = plot_data_2d(text=False, specific_cat='Weapons')

In [17]:
#
# Scree plot
#

def scree(dim=1000, threshold=.9):
    
    if dim >= input_data.shape[1]:
        dim = int(input_data.shape[1]/2)
        
    xRed_, pca2 = red_dim(input_data, dim)


    plt.ylim(0)
#     plt.text(1,0.85,"Number of components\n"+str(input_data.shape[1]))
    var_rat = np.cumsum(pca2.explained_variance_ratio_)
    var = np.cumsum(pca2.explained_variance_)
    var = [float(i)/pca2.explained_variance_[0] for i in pca2.explained_variance_]

    %matplotlib inline
    matplotlib.rcParams.update({'font.size': 16})
    plt.ylabel("Variance")
    plt.xlabel("Components")
    plt.title("{}\n{} Features".format(domain, input_data.shape[1]))
    plt.plot(var_rat, label="Cumulative variance ratio")
    plt.plot(var, label="Normalized variance")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.ylim([0,1])
    plt.show()
    
    for i, x in enumerate(var_rat):
        if x > threshold:
            return i
    return dim
        
# scree(100, .95)

In [18]:
#
# Make same domain etc.
#

# description["poseidon"].append('Services sad sd')
# description["poseidon"].append('Jewels&Gold sad sd')
# category["poseidon"].append('Services')
# category["poseidon"].append('Jewels&Gold')

pprint(len(set(category["poseidon"])))
pprint(len(set(category["alphabay"])))

10
12


In [19]:
domain = "poseidon"
problem = category[domain]
input_data = tfidf(description[domain])
problem_numeric = [sorted(list(set(category[domain]))).index(x) for x in problem]

print("Samples:", len(problem))
print("Features:", input_data.shape[1])

Samples: 3688
Features: 250000


In [22]:
##
## Problem and target 
##

domain = "poseidon"
problem = category[domain]
input_data = tfidf(description[domain])
problem_numeric = [sorted(list(set(category[domain]))).index(x) for x in problem]

print("Samples:", len(problem))
print("Features:", input_data.shape[1])


##
## Visualize data
##
# plot_data_3d()
# plot_data_2d()

##
## Check BOW-model feature restraints
##

# feat_impact(domain, start=10, stop=2500, step=100)

##
## Check when exp. var > treshold (nm of useful features) 
##

# dims = scree(3000, .99)
# dims = 500

##
## Train models 
##

# clf = create_clf(input_data)
# clfD = (clf, input_data, "none", "{} - Default".format(input_data.shape[1]))
# clfs = [clfD]

clfs = []
# for x in tqdm([int(dims), int(dims/2)]):
# #     input_, pca = red_dim(input_data, x , domain_bypass='poseidon')
#     input_, pca = lda(input_data) #, domain_bypass='poseidon')
#     clf = create_clf(input_) #, domain_bypass='poseidon')
#     clfs.append((clf, input_, pca, "{}dim".format(x)))

input_, pca = lda(input_data) #, domain_bypass='poseidon')
clf = create_clf(input_) #, domain_bypass='poseidon')
clfs.append((clf, input_, pca, "{}dim".format("3000 Intermediate")))

##
## Display result
##

plot_clfs(clfs, best_line=True)


##
## Result F1 & Confusion matrix
##

for clf_ in clfs:
    eval_clf(clf_)
    plot_confusion_matrix(clf_, normalize=True)


Samples: 3688
Features: 150000


MemoryError: 

In [ ]:
plot_confusion_matrix(clfs[-1], normalize=True, estimator=-1)
plot_confusion_matrix(clfs[-1], normalize=False, estimator=-1)

plot_confusion_matrix(clfs[0], normalize=True, estimator=-1)
plot_confusion_matrix(clfs[0], normalize=False, estimator=-1)